In [1]:
spark

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.0.7:4040
SparkContext available as 'sc' (version = 3.1.2, master = local[*], app id = local-1640601896826)
SparkSession available as 'spark'


res0: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@1da3a7fd


In [2]:
val myRange = spark.range(1000).toDF("number")

myRange: org.apache.spark.sql.DataFrame = [number: bigint]


# 2.7 트랜스포메이션
DataFrame을 변경하려면 원하는 변경 방법을 스파크에 알려줘야 함<br/>
이때 사용하는 명령을 transformation이라고 함<br/>
다음 코드는 DataFrame에서 짝수를 찾는 간단한 transformation 예제임<br/>

In [3]:
val divisBy2 = myRange.where("number % 2 = 0")

divisBy2: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [number: bigint]


추상적인 transformaiton만 지정한 상태이기 때문에 위 코드를 실행해도 결과는 출력되지 않음<br/>
실제 transformation을 수행하기 위해서는 action을 호출해야 함<br/>
<br/>
transformation에는 두 가지 유형이 있음<br/>
1. 좁은 의존성 (narrow dependency)
2. 넓은 의존성 (wide dependency)<br/>


좁은 의존성을 가진 transformation은 각 입력 파티션이 하나의 출력 파티션에만 영향을 미침<br/>
where 구문은 좁은 의존성을 가짐<br/>
넓은 의존성을 가진 transformation은 하나의 입력 파티션이 여러 출력 파티션에 영향을 미침 ex) shuffle <br/>

좁은 transformation을 사용하면 스파크에서 파이프라이닝을 자동으로 수행함<br/>
즉, DataFrame에 여러 필터를 지정하는 경우 모든 작업이 메모리에서 일어남<br/>
하지만 셔플은 다른 방식으로 동작함<br/>
스파크는 셔플의 결과를 디스크에 저장함<br/>

# 2.8 액션
action은 일련의 transformation으로부터 결과를 계산하도록 지시하는 명령임<br/>
가장 단순한 액션인 count 메서드는 DataFrame의 전체 레코드 수를 반환함<br/>

action을 지정하면 스파크 job이 시작됨<br/>
스파크 job은 filter(좁은 transformation)를 수행한 후 파티션별로 레코드 수를 count(넓은 transformation)함<br/>
그리고 각 언어에 적합한 네이티브 객체에 결과를 모음<br/>

In [4]:
divisBy2.count()

res1: Long = 500


# 2.10 종합 예제
데이터는 SparkSession의 DataFrameReader 클래스를 사용해서 읽음<br/>
이때 특정 파일 포맷과 몇 가지 옵션을 함께 설정함<br/>
예제에서는 스파크 DataFrame의 스키마 정보를 알아내는 스키마 추론(schema inference) 기능을 사용함<br/>

스파크는 스키마 정보를 얻기 위해 데이터를 조금 읽고 해당 로우의 데이터 타입을 스파크 데이터 타입에 맞게 분석함<br/>
하지만 운영 환경에서는 데이터를 읽는 시점에 스키마를 엄격하게 지정하는 옵션을 사용해야 함<br/>

read 메서드는 좁은 transformation에 해당함

In [11]:
val flightData2015 = spark
    .read
    .option("inferSchema", "true")
    .option("header", "true")
    .csv("Downloads/Spark-The-Definitive-Guide/data/flight-data/csv/2015-summary.csv")

flightData2015: org.apache.spark.sql.DataFrame = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]


In [12]:
flightData2015.take(3)

res7: Array[org.apache.spark.sql.Row] = Array([United States,Romania,15], [United States,Croatia,1], [United States,Ireland,344])


sort 메서드는 넓은 transformation에 해당하며, DataFrame을 변경하지 않음<br/>
transformation으로 sort 메서드를 사용하면 이전의 DataFrame을 변환해 새로운 DataFrame을 생성해 반환함<br/>

sort 메서드는 단지 transformation이기 때문에 호출 시 데이터에 아무런 변화도 일어나지 않음<br/>
하지만 스파크는 실행 계획을 만들고 검토하여 클러스터에서 처리할 방법을 알아냄<br/>
특정 DataFrame 객체에 explain 메서드를 호출하면 DataFrame의 계보(lineage)나 스파크의 쿼리 실행 계획을 확인할 수 있음<br/>

In [13]:
flightData2015.sort("count").explain()

== Physical Plan ==
*(1) Sort [count#28 ASC NULLS FIRST], true, 0
+- Exchange rangepartitioning(count#28 ASC NULLS FIRST, 200), ENSURE_REQUIREMENTS, [id=#57]
   +- FileScan csv [DEST_COUNTRY_NAME#26,ORIGIN_COUNTRY_NAME#27,count#28] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/home/hj/Downloads/Spark-The-Definitive-Guide/data/flight-data/csv/2015-su..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:int>




스파크는 shuffle 수행 시 기본적으로 200개의 shuffle 파티션을 생성함<br/>
이 값을 5로 설정해 shuffle의 출력 파티션 수를 줄임<br/>

In [15]:
spark.conf.set("spark.sql.shuffle.partitions", "5")
flightData2015.sort("count").take(2)

res10: Array[org.apache.spark.sql.Row] = Array([United States,Singapore,1], [Moldova,United States,1])


transformation의 논리적 실행 계획은 DataFrame의 계보를 정의함<br/>
스파크는 lineage를 통해 입력 데이터에 수행한 연산을 전체 파티션에서 어떻게 재연산하는지 알 수 있음<br/>
이 기능은 스파크의 프로그래밍 모델인 함수형 프로그래밍의 핵심임<br/>
함수형 프로그래밍은 데이터의 변환 규칙이 일정한 경우 같은 입력에 대해 항상 같은 출력을 생성함<br/>

사용자는 물리적 데이터를 직접 다루지 않는 대신 앞서 설정한 shuffle 파티션 파라미터와 같은 속성으로 물리적 실행 특성을 제어함<br/>
앞서 shuffle 파티션 수를 5로 설정했는데, 이 값을 변경하면 런타임이 크게 달라질 수 있음<br/>
스파크 UI에서 job의 실행 상태와 물리적, 논리적 실행 특성을 확인할 수 있음 <br/>

## 2.10.1 DataFrame과 SQL
사용자가 SQL이나 DataFrame(R, 파이썬, 스칼라, 자바에서)으로 비즈니스 로직을 표현하면 스파크에서 실제 코드 실행 이전에 그 로직을 기본 실행 계획(explain 메서드 호출을 통해 확인 가능)으로 컴파일함<br/>
스파크 SQL을 사용하면 모든 DataFrame을 테이블이나 뷰(임시 테이블)로 등록한 후 SQL 쿼리를 사용할 수 있음<br/>
스파크는 SQL 쿼리를 DataFrame 코드와 같은 실행 계획으로 컴파일하므로 둘 사이의 성능 차이는 없음<br/>

다음 예제처럼 createOrReplaceTempView 메서드를 호출하면 모든 DataFrame을 테이블이나 뷰로 만들 수 있음<br/>

In [16]:
flightData2015.createOrReplaceTempView("flight_data_2015")

이제 SQL로 데이터를 조회할 수 있음<br/>
새로운 DataFrame을 반환하는 spark.sql 메서드로 SQL 쿼리를 실행함<br/>
spark는 SparkSession의 변수임<br/>
DataFrame에 쿼리를 수행하면 새로운 DataFrame을 반환함<br/>

이해를 돕기 위해 두 가지 실행 계획을 비교해보겠음<br/>

In [17]:
val sqlWay = spark.sql("""
SELECT DEST_COUNTRY_NAME, count(1)
FROM flight_data_2015
GROUP BY DEST_COUNTRY_NAME
""")

val dataFrameWay = flightData2015
    .groupBy("DEST_COUNTRY_NAME")
    .count()

sqlWay.explain
dataFrameWay.explain

== Physical Plan ==
*(2) HashAggregate(keys=[DEST_COUNTRY_NAME#26], functions=[count(1)])
+- Exchange hashpartitioning(DEST_COUNTRY_NAME#26, 5), ENSURE_REQUIREMENTS, [id=#86]
   +- *(1) HashAggregate(keys=[DEST_COUNTRY_NAME#26], functions=[partial_count(1)])
      +- FileScan csv [DEST_COUNTRY_NAME#26] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/home/hj/Downloads/Spark-The-Definitive-Guide/data/flight-data/csv/2015-su..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>


== Physical Plan ==
*(2) HashAggregate(keys=[DEST_COUNTRY_NAME#26], functions=[count(1)])
+- Exchange hashpartitioning(DEST_COUNTRY_NAME#26, 5), ENSURE_REQUIREMENTS, [id=#105]
   +- *(1) HashAggregate(keys=[DEST_COUNTRY_NAME#26], functions=[partial_count(1)])
      +- FileScan csv [DEST_COUNTRY_NAME#26] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/home/hj/Downloads/Spark-The-Definitive-Guide/data/flight-data/csv

sqlWay: org.apache.spark.sql.DataFrame = [DEST_COUNTRY_NAME: string, count(1): bigint]
dataFrameWay: org.apache.spark.sql.DataFrame = [DEST_COUNTRY_NAME: string, count: bigint]


두 가지 실행 계획이 동일한 기본 실행 계획으로 컴파일되는 것을 확인할 수 있음

스파크는 빅데이터 문제를 빠르게 해결하는 데 필요한 수백 개의 함수를 제공함<br/>
다음 예제에서는 특정 위치를 왕래하는 최대 비행 횟수를 구하기 위해 max 함수를 사용함<br/>
max 함수는 DataFrame의 특정 컬럼 값을 스캔하면서 이전 최댓값보다 더 큰 값을 찾음<br/>
max 함수는 필터링을 수행해 단일 로우를 결과로 반환하는 transformation임<br/>

In [18]:
spark.sql("SELECT max(count) from flight_data_2015").take(1)

res13: Array[org.apache.spark.sql.Row] = Array([370002])


In [19]:
import org.apache.spark.sql.functions.max

flightData2015.select(max("count")).take(1)

import org.apache.spark.sql.functions.max
res14: Array[org.apache.spark.sql.Row] = Array([370002])


다음 예제에서는 상위 5개의 도착 국가를 찾아내는 복잡한 코드를 실행해보겠음<br/>
이 예제는 처음으로 다뤄보는 다중 transformation 쿼리이기 때문에 단계별로 알아보겠음<br/>
먼저 아주 직관적인 SQL 집계 쿼리부터 시작함<br/>

In [23]:
val maxSql = spark.sql("""
SELECT DEST_COUNTRY_NAME, sum(count) as destination_total
FROM flight_data_2015
GROUP BY DEST_COUNTRY_NAME
ORDER BY sum(count) DESC
LIMIT 5
""")

maxSql.show()

+-----------------+-----------------+
|DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
|    United States|           411352|
|           Canada|             8399|
|           Mexico|             7140|
|   United Kingdom|             2025|
|            Japan|             1548|
+-----------------+-----------------+



maxSql: org.apache.spark.sql.DataFrame = [DEST_COUNTRY_NAME: string, destination_total: bigint]


이어서 의미는 같고 구현과 정렬 방식이 조금 다른 DataFrame 구문을 살펴보겠음<br/>
앞서 설명했듯 기본 실행 계획은 SQL로 정의한 처리와 같음<br/>

In [24]:
import org.apache.spark.sql.functions.desc

flightData2015
    .groupBy("DEST_COUNTRY_NAME")
    .sum("count")
    .withColumnRenamed("sum(count)", "destination_total")
    .sort(desc("destination_total"))
    .limit(5)
    .show()

+-----------------+-----------------+
|DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
|    United States|           411352|
|           Canada|             8399|
|           Mexico|             7140|
|   United Kingdom|             2025|
|            Japan|             1548|
+-----------------+-----------------+



import org.apache.spark.sql.functions.desc


In [25]:
maxSql.explain

== Physical Plan ==
TakeOrderedAndProject(limit=5, orderBy=[destination_total#79L DESC NULLS LAST], output=[DEST_COUNTRY_NAME#26,destination_total#79L])
+- *(2) HashAggregate(keys=[DEST_COUNTRY_NAME#26], functions=[sum(cast(count#28 as bigint))])
   +- Exchange hashpartitioning(DEST_COUNTRY_NAME#26, 5), ENSURE_REQUIREMENTS, [id=#232]
      +- *(1) HashAggregate(keys=[DEST_COUNTRY_NAME#26], functions=[partial_sum(cast(count#28 as bigint))])
         +- FileScan csv [DEST_COUNTRY_NAME#26,count#28] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/home/hj/Downloads/Spark-The-Definitive-Guide/data/flight-data/csv/2015-su..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,count:int>




In [26]:
flightData2015
    .groupBy("DEST_COUNTRY_NAME")
    .sum("count")
    .withColumnRenamed("sum(count)", "destination_total")
    .sort(desc("destination_total"))
    .limit(5)
    .explain

== Physical Plan ==
TakeOrderedAndProject(limit=5, orderBy=[destination_total#128L DESC NULLS LAST], output=[DEST_COUNTRY_NAME#26,destination_total#128L])
+- *(2) HashAggregate(keys=[DEST_COUNTRY_NAME#26], functions=[sum(cast(count#28 as bigint))])
   +- Exchange hashpartitioning(DEST_COUNTRY_NAME#26, 5), ENSURE_REQUIREMENTS, [id=#256]
      +- *(1) HashAggregate(keys=[DEST_COUNTRY_NAME#26], functions=[partial_sum(cast(count#28 as bigint))])
         +- FileScan csv [DEST_COUNTRY_NAME#26,count#28] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/home/hj/Downloads/Spark-The-Definitive-Guide/data/flight-data/csv/2015-su..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,count:int>




실행 계획은 transformation의 DAG이며, action이 호출되면 결과를 만들어냄<br/>
그리고 DAG의 각 단계는 불변성을 가진 신규 DataFrame을 생성함<br/>

# 2.11 정리
이 장에서는 transformation과 action, 그리고 스파크가 DataFrame의 실행 계획을 최적화하기 위해 transformation의 DAG를 지연 실행하는 방법을 설명했음<br/>
그리고 데이터가 파티션으로 구성되는 방법과 복잡한 transformation 작업을 실행하는 단계도 함께 알아보았음<br/>